Stage.OpenMaskedは、その名の通り PopulationMaskで指定したSdfPathに含まれるPrimのみ  
ロードすることができる機能。  
巨大なシーンのうち、一部だけ（レンダーセッティングだけ、ある区画のモデルだけ）を編集したい場合  
すべてをロードするのではなく、Maskを使用して部分的にロードすることで  
より効率的に作業することができる。

In [7]:
import os
os.makedirs('usd')
ROOT_DIR = os.getcwd() + "/usd"

In [4]:
from pxr import Usd,Sdf

In [9]:
stage = Usd.Stage.CreateInMemory()
stage.DefinePrim("/a")
stage.DefinePrim("/b")
stage.DefinePrim("/c")
stage.GetRootLayer().Export(f"{ROOT_DIR}/loadLayer.usd")

True

In [18]:
# SdfPathで指定した部分だけロードするのがOpenMasked
# SdfPathのリストを作るのがStagePopulationMask
mask = Usd.StagePopulationMask().Add('/a')

In [15]:
stageB = Usd.Stage.OpenMasked(f"{ROOT_DIR}/loadLayer.usd",mask)

In [17]:
# /a のみロードするようにMaskした例。
print(stageB.ExportToString())

#usda 1.0
(
    doc = """Generated from Composed Stage of root layer /work_dir/usd/loadLayer.usd
"""
)

def "a"
{
}




In [20]:
# ２つのPopulationMaskを結合
maskB = Usd.StagePopulationMask().Add('/b')
union = Usd.StagePopulationMask().Union(mask,maskB)
print(union)

UsdStagePopulationMask([ /a /b ])


In [32]:
# ２つのPopulationMaskの重複部分を取得する場合
intersection = Usd.StagePopulationMask().Intersection(mask,maskB)
print(intersection)
# PopulationMaskにPathが含まれてるか？
print(intersection.IsEmpty())

UsdStagePopulationMask([ ])
True


In [25]:
stageC = Usd.Stage.OpenMasked(f"{ROOT_DIR}/loadLayer.usd", mask.GetUnion(maskB))
print(stageC.ExportToString())

#usda 1.0
(
    doc = """Generated from Composed Stage of root layer /work_dir/usd/loadLayer.usd
"""
)

def "a"
{
}

def "b"
{
}




In [26]:
mask.Includes(maskB)

False

In [29]:
maskC = Usd.StagePopulationMask().Add('/a')

In [31]:
# maskに maskC の要素が含まれるかどうか
mask.Includes(maskC)

True

In [33]:
# Get～だと、今のMaskと引数のMaskの結果を取得できる
print(mask.GetUnion(maskB))

UsdStagePopulationMask([ /a /b ])
